# Практика к лекции 2, часть 2.
### Очистка текста и векторизация с помощью Word2Vec

In [1]:
import os
import re
import string
import annoy
import codecs
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

tqdm.pandas()

## 1. Загрузка данных
Для корректного запуска кода данные (датасет ProductsDataset.csv) должен находиться в папке с названием data. Также необходимо создать папку storage, куда сохранится обученная модель и индекс.

In [2]:
DATADIR = 'data'
STORAGEDIR = 'storage'

In [3]:
# Загружаем датасет товаров
with codecs.open('/'.join([DATADIR, 'ProductsDataset.csv']), 'r', 'utf-8') as file:
    data = pd.read_csv(file)
data.head()    

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [4]:
# Исправляем ошибки в наименовании колонки
data.rename(columns={'descrirption': 'description'}, inplace=True)

## 2. Очистка текста

In [5]:
exclude = set(string.punctuation) # удаление пунктуации
stop_w = set(get_stop_words(language='ru')) # удаление стоп-слов
morpher = MorphAnalyzer() # леммантизатор

In [6]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [7]:
print(get_stop_words(language='ru'))

['а', 'в', 'г', 'е', 'ж', 'и', 'к', 'м', 'о', 'с', 'т', 'у', 'я', 'бы', 'во', 'вы', 'да', 'до', 'ее', 'ей', 'ею', 'её', 'же', 'за', 'из', 'им', 'их', 'ли', 'мы', 'на', 'не', 'ни', 'но', 'ну', 'нх', 'об', 'он', 'от', 'по', 'со', 'та', 'те', 'то', 'ту', 'ты', 'уж', 'без', 'был', 'вам', 'вас', 'ваш', 'вон', 'вот', 'все', 'всю', 'вся', 'всё', 'где', 'год', 'два', 'две', 'дел', 'для', 'его', 'ему', 'еще', 'ещё', 'или', 'ими', 'имя', 'как', 'кем', 'ком', 'кто', 'лет', 'мне', 'мог', 'мож', 'мои', 'мой', 'мор', 'моя', 'моё', 'над', 'нам', 'нас', 'наш', 'нее', 'ней', 'нем', 'нет', 'нею', 'неё', 'них', 'оба', 'она', 'они', 'оно', 'под', 'пор', 'при', 'про', 'раз', 'сам', 'сих', 'так', 'там', 'тем', 'тех', 'том', 'тот', 'тою', 'три', 'тут', 'уже', 'чем', 'что', 'эта', 'эти', 'это', 'эту', 'алло', 'буду', 'будь', 'бывь', 'была', 'были', 'было', 'быть', 'вами', 'ваша', 'ваше', 'ваши', 'ведь', 'весь', 'вниз', 'всем', 'всех', 'всею', 'года', 'году', 'даже', 'двух', 'день', 'если', 'есть', 'зато', 'ко

In [8]:
# Компилируем шаблоны для быстрой обработки
# спецсимволы и emoji
symbols_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "@_!#$%^&*()<>?/\|}{~:√•"
                       "]+", flags = re.UNICODE)
# двойные пробелы
space_pattern = re.compile('\s+')

In [9]:
def clear_text(text):
    """ Функция удаления спецсимволов"""
    # удаление спецсимволов и emoji
    pre = symbols_pattern.sub(r'',text)
    # удаление двойных пробелов
    
    return space_pattern.sub(' ', pre)

In [10]:
def preprocess_text(text):
    """ Финальная функция для обработки """
    # srip + lower + punctuation
    sentence = (
        ''.join([x for x in str(text).strip().lower() if x not in exclude])
    )
    # лемматизация и стопслова
    sentence = (
        ' '.join([
            morpher.parse(word)[0].normal_form 
            for word in sentence.split() 
            if word not in stop_w])
    )
    
    return clear_text(sentence)

In [11]:
# Обработаем title
data['title'] = data['title'].progress_apply(preprocess_text)
data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,description,product_id,category_id,subcategory_id,properties,image_links
0,юбка детский orby,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,ботильон,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,продать детский шапка,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [12]:
# Обработаем description
data['description'] = data['description'].progress_apply(preprocess_text)
data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,description,product_id,category_id,subcategory_id,properties,image_links
0,юбка детский orby,новый носить раз реал красивый фото,58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,ботильон,новыепривезти чехия указать размер 40ный малом...,5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,брюки,размер 4042 брюки новый знать мерило покупка н...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,продать детский шапка,продать шапкикажда 200ррозовый белый проданна,57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,блузка,темносиний 42 размерсостояние отличноекак новы...,5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


## 3. Обучение Word2Vec

In [13]:
# используем столбцы title и description
sentences = []
for sentence in tqdm(pd.Series(data['title'] + ' ' + data['description']).to_list()):
    sentences.append(sentence.split())

  0%|          | 0/35548 [00:00<?, ?it/s]

In [14]:
sentences = [x for x in sentences if len(x)>2]
model_shop = Word2Vec(sentences=sentences, min_count=1, window=5)
model_shop.save('/'.join([STORAGEDIR, 'w2v_model_shop']))

In [15]:
def get_vector(sent, model):
    """ Функция возвращает вектор предложения """
    n_w2v = 0
    vector = np.zeros(100)
    for word in sent:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return vector

## 4. Построение индекса
### Очень важно для быстрого поиска нужных векторов

FAISS

In [16]:
# инициализируем индексатор
index = annoy.AnnoyIndex(100, 'angular')

index_map = {}
counter = 0

# упаковываем в индекс 'title', 'description', 'product_id'
for line in data[['title', 'description', 'product_id']].values:
    index_map[counter] = [line[0], line[1], line[2]]
    
    title = line[0].split()
    index.add_item(counter, get_vector(title, model_shop)) 
    counter += 1

index.build(10)
index.save('/'.join([STORAGEDIR, 'idx.ann']))

True

## PS:
* Bag-of-Wordsv реализован в классе [CountVectorizer()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
* Tf-Idf в [TfidfVectorizer()](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

!!! Не забывайте, что правильнее делать fit_transform на обучающих данных, а на тестовых только transform:)